In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel, RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from dataset import Downstream_Dataset, DataAugmentation, LoadPretrainData
import torch
import torch.nn as nn
from torchmetrics import R2Score
from torch.utils.tensorboard import SummaryWriter
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data = pd.read_csv('data/practice.csv')
len(train_data)

19

In [6]:
scaler = StandardScaler()
train_data.iloc[:, 1] = scaler.fit_transform(train_data.iloc[:, 1].values.reshape(-1, 1))
train_data

,smiles,value
0,*CC(*)C,0.673820
1,*CC(*)CC,0.787752
2,*CC(*)CCC,0.431143
3,*CC1CCC(*)C1,0.826822
4,*CC(*)CCCC1CCCCC1,0.601899
5,*C(*)C,0.507022
6,*C=C*,-3.138791
7,*C=CCCCC*,-0.402452
8,*C1CCC1*,0.019180
9,*CC1(*)CCCCC1,0.469704


In [7]:
PretrainedModel = RobertaModel.from_pretrained('ckpt/pretrain.pt')
tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=411)
train_dataset = Downstream_Dataset(train_data, tokenizer, 411)
train_dataset[1]

OSError: You seem to have cloned a repository without having git-lfs installed. Please install git-lfs and run `git lfs install` followed by `git lfs pull` in the folder you cloned.

In [8]:
class DownstreamRegression(nn.Module):
    def __init__(self, drop_rate=0.1):
        super(DownstreamRegression, self).__init__()
        self.PretrainedModel = deepcopy(PretrainedModel)
        self.PretrainedModel.resize_token_embeddings(len(tokenizer))
        
        self.Regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(self.PretrainedModel.config.hidden_size, self.PretrainedModel.config.hidden_size),
            nn.SiLU(),
            nn.Linear(self.PretrainedModel.config.hidden_size, 1)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.PretrainedModel(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.last_hidden_state[:, 0, :] #fingerprint
        print(type(logits))
        output = self.Regressor(logits)
        return output

In [9]:
def train(model, optimizer, scheduler, loss_fn, train_dataloader, device):
    model.eval()

    for step, batch in enumerate(train_dataloader):
        print(f'Smile: {step+1}')
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        prop = batch["prop"].to(device).float()
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask).float()
        print(outputs)
        

    return None

In [10]:
model = DownstreamRegression(drop_rate=0.1).to(device)
model = model.double()
loss_fn = nn.MSELoss()

NameError: name 'PretrainedModel' is not defined

In [ ]:
train_dataloader = DataLoader(train_dataset, 1, shuffle=False, num_workers=8)

In [ ]:
"""Parameters for scheduler"""
steps_per_epoch = train_data.shape[0] // 1
training_steps = steps_per_epoch * 1
warmup_steps = int(training_steps * 0.05)

optimizer = AdamW(
                    [
                        {"params": model.PretrainedModel.parameters(), "lr":  0.00005,
                         "weight_decay": 0.0},
                        {"params": model.Regressor.parameters(), "lr": 0.0001,
                         "weight_decay": 0.01},
                    ],
    				no_deprecation_warning=True
                )
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                        num_training_steps=training_steps)

In [ ]:
for epoch in range(1):
    print("Training epoch: %s/%s" % (epoch+1, 1))
    train(model, optimizer, scheduler, loss_fn, train_dataloader, device)

Training epoch: 1/1
Smile: 1
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 2
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 3
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 4
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 5
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 6
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 7
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 8
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 9
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 10
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 11
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 12
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 13
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 14
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 15
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 16
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 17
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 18
<class 'to

<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 145
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 146
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 147
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 148
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 149
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 150
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 151
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 152
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 153
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 154
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 155
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 156
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 157
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 158
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 159
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 160
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 161
<class 'torch.

<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 287
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 288
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 289
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 290
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 291
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 292
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 293
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 294
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 295
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 296
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 297
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 298
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 299
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 300
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 301
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 302
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 303
<class 'torch.

<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 429
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 430
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 431
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 432
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 433
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 434
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 435
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 436
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 437
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 438
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 439
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 440
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 441
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 442
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 443
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 444
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 445
<class 'torch.

<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 571
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 572
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 573
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 574
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 575
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 576
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 577
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 578
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 579
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 580
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 581
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 582
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 583
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 584
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 585
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 586
<class 'torch.Tensor'>
<class 'torch.Tensor'>

Smile: 587
<class 'torch.

KeyboardInterrupt: 